In [12]:
import mcdm
import numpy as np
from pymcdm.methods import TOPSIS
from pymcdm.methods import VIKOR
from pymcdm.methods import PROMETHEE_II
from pymcdm.helpers import rankdata,rrankdata
from pymcdm import weights as mcdm_weights

In [13]:
from prettytable import from_csv
with open("hep_perform.tsv") as fp:
    mytable = from_csv(fp)
print(mytable)

+----------------------+---------+-----------+--------+-------+----------+---------+---------+-------------+--------------+--------------+
|                      | ACURACY | PRECISION | RECALL |  AUC  | F1-SCORE | FP_RATE | FN_RATE | SPECIFICITY | HAMMING LOSS | ENTROPY LOSS |
+----------------------+---------+-----------+--------+-------+----------+---------+---------+-------------+--------------+--------------+
| LOGISTIC REGRESSION  |   0.86  |    0.89   |  0.86  |  0.86 |   0.87   |   0.12  |   0.25  |     0.88    | 0.137931034  | 4.764051875  |
|    DECISION TREE     |   0.79  |    0.87   |  0.79  | 0.775 |   0.82   |   0.2   |   0.25  |     0.8     | 0.206896552  | 7.146091599  |
| GAUSSIAN NAIVE BAYES |   0.83  |    0.88   |  0.83  |  0.95 |   0.85   |   0.16  |   0.25  |     0.84    | 0.172413793  | 5.955071737  |
|         SVM          |   0.86  |    0.74   |  0.86  |  0.63 |   0.8    |    0    |    1    |      1      | 0.137931034  | 4.763969158  |
|    NEURAL NETWORK    |   

In [14]:
x_matrix, alt_names = mcdm.load("hep_perform.tsv", delimiter="\t", skiprows=1, labeled_rows=True)
alts = np.array(x_matrix, dtype='float')
weights = mcdm_weights.equal(alts)
weight=weights.tolist()
types = np.array([1,1,1,1,1,-1,-1,1,-1,-1])

In [15]:
topsis = TOPSIS()
# Determine preferences and ranking for alternatives
pref1 = topsis(alts, weights, types)
ranking1= rrankdata(pref1)

for r, p in zip(ranking1, pref1):
    print(r, p)


2.0 0.7969103845723764
5.0 0.4985724196076565
3.0 0.678156873084089
4.0 0.6116320148255644
7.0 0.3475318192132489
1.0 0.8131457124040566
6.0 0.3488284877446559


In [16]:
vikor= VIKOR()
pref2 = vikor(alts, weights, types)
ranking2= rankdata(pref2)

for r, p in zip(ranking2, pref2):
    print(r, p)

2.0 0.12499999999999994
5.0 0.8134292550523017
3.0 0.3765622679251991
4.0 0.6571591480124447
6.0 0.989530369969404
1.0 0.01996437411488836
7.0 1.0


In [17]:
promethe=PROMETHEE_II('usual')
p = np.random.rand(x_matrix.shape[1]) / 2
q = np.random.rand(x_matrix.shape[1]) / 2 + 0.5
pref3 = promethe(alts, weights, types,p=p,q=q)
ranking3 = rrankdata(pref3)

for r, p in zip(ranking3, pref3):
    print(r, p)

1.5 0.5
5.0 -0.23333333333333328
4.0 0.13333333333333325
3.0 0.21666666666666673
6.0 -0.4999999999999998
1.5 0.5
7.0 -0.6166666666666666


In [18]:
from prettytable import PrettyTable
print("#########################BREAST DATASET#######################")
columns = ["Model", "TOPSIS","Value","VIKOR","VALUES","PROMETHEE","VALUES"]
Table = PrettyTable()
Table.add_column(columns[0], ["LOGISTIC REGRESSION ", "DECISION TREE", "GAUSSIAN NAIVE BAYES",
                       "SVM ", "NEURAL NETWORK", "RANDOM FOREST ", "LINEAR MODEL"])
Table.add_column(columns[1], ranking1)
Table.add_column(columns[2], pref1)
Table.add_column(columns[3], ranking2)
Table.add_column(columns[4], pref2)
Table.add_column(columns[5], ranking3)
Table.add_column(columns[6], pref3)

print(Table)

#########################BREAST DATASET#######################
+----------------------+--------+--------------------+-------+---------------------+-----------+----------------------+
|        Model         | TOPSIS |       Value        | VIKOR |        VALUES       | PROMETHEE |        VALUES        |
+----------------------+--------+--------------------+-------+---------------------+-----------+----------------------+
| LOGISTIC REGRESSION  |  2.0   | 0.7969103845723764 |  2.0  | 0.12499999999999994 |    1.5    |         0.5          |
|    DECISION TREE     |  5.0   | 0.4985724196076565 |  5.0  |  0.8134292550523017 |    5.0    | -0.23333333333333328 |
| GAUSSIAN NAIVE BAYES |  3.0   | 0.678156873084089  |  3.0  |  0.3765622679251991 |    4.0    | 0.13333333333333325  |
|         SVM          |  4.0   | 0.6116320148255644 |  4.0  |  0.6571591480124447 |    3.0    | 0.21666666666666673  |
|    NEURAL NETWORK    |  7.0   | 0.3475318192132489 |  6.0  |  0.989530369969404  |    6.0    | 

In [19]:
ans=mcdm.rank(x_matrix, alt_names=alt_names, n_method="Linear2",is_benefit_x=[1,1,1,1,1,-1,-1,1,-1,-1], w_vector=weight, s_method="SAW") 
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################SAW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table) 

#########################SAW#######################
+----------------------+--------------------+
|        Model         |       Value        |
+----------------------+--------------------+
| GAUSSIAN NAIVE BAYES | 0.6105099783245274 |
|    RANDOM FOREST     | 0.5912047511312217 |
| LOGISTIC REGRESSION  | 0.582356413584042  |
|    DECISION TREE     | 0.5647247367732655 |
|         SVM          | 0.4896587481146305 |
|     LINEAR MODEL     | 0.3359728506787331 |
|    NEURAL NETWORK    | 0.3321428575053572 |
+----------------------+--------------------+


In [20]:
ans=mcdm.rank(x_matrix,alt_names=alt_names,n_method="Linear1",is_benefit_x=[1,1,1,1,1,-1,-1,1,-1,-1], w_vector=weight,s_method="MEW")
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################MEW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table)

#########################MEW#######################
+----------------------+--------------------+
|        Model         |       Value        |
+----------------------+--------------------+
|     LINEAR MODEL     | 0.7891068884737424 |
|    DECISION TREE     | 0.7760986425318048 |
| GAUSSIAN NAIVE BAYES | 0.761548924971407  |
|    NEURAL NETWORK    | 0.7605654215040737 |
|    RANDOM FOREST     | 0.7328903838809628 |
| LOGISTIC REGRESSION  | 0.7113847383739225 |
|         SVM          |        0.0         |
+----------------------+--------------------+
